### Cargamos Ontologia y Vocabulario

In [112]:
from ontologia import *

In [7]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../Scripts')
import pandas as pd 
from ontologia import *
import ast
import difflib
#Frequency
from collections import Counter 
from itertools import repeat, chain 

#ElasticSearch
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec
from time import time

from numpy import (
    dot, float32 as REAL
)
import numpy as np
from elasticsearch import Elasticsearch


#Lectura de lemas 
import json
with open('./Vocabulario/diccionario-lemas.json', 'r') as f:
    diccionario_lemas = json.loads(f.read())

lemas = list(diccionario_lemas.keys())

es = Elasticsearch()
es.ping()

True

In [8]:
modeloD2V = Doc2Vec.load("./Modelos/modelo2_FINALD2V.model")

**Palabras a Buscar**

In [9]:
words = ['gestor', 'conocimiento']

#### Consulta de Palabras mas similares y en contexto

In [28]:
#Retrive Vectors
keywords = []
for word in words:
    term = {"term": {"palabra": ""}}
    term['term']['palabra'] = word
    keywords.append(term)


#Aqui Añadimos las palabras
query_vecs = {
    "query":{
        "bool":{
            "should":keywords
        }
    },
    "_source": ["palabra", "norm_vec"]
}
#Calculo del Vector
vectores = es.search(index='palabras', body=query_vecs)['hits']['hits']
norm_vecs = [vector['_source']['norm_vec'] for vector in vectores]
vector_mean = np.array(norm_vecs).mean(axis=0)
unit_vec = vector_mean / np.linalg.norm(vector_mean)

#Most similar Words
query_sims = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec
        }
      }
    }
  },
  "size": 20,
  "_source": ["palabra", "index"]
}
most_similar_words = es.search(index='palabras', body=query_sims)['hits']['hits']
most_similar_words = [(palabra['_source']['palabra'], palabra['_score']) for palabra in most_similar_words]
most_similar_words

[('gestor', 0.81737363),
 ('conocimiento', 0.8173736),
 ('debilmente', 0.77090985),
 ('umayux', 0.74467164),
 ('sgbd', 0.7438246),
 ('sparql', 0.7421229),
 ('rdf', 0.71533084),
 ('postgresql', 0.7092361),
 ('ontologias', 0.707866),
 ('fundamentandose', 0.69965184),
 ('solicitada', 0.690233),
 ('maskana', 0.6835666),
 ('motor', 0.6802874),
 ('transformarla', 0.677389),
 ('facilite', 0.66182),
 ('adquieran', 0.66137993),
 ('buscador', 0.6595822),
 ('digitalizados', 0.6512728),
 ('flock', 0.6399753),
 ('robusto', 0.6398908)]

#### Consulta en Ontologia

In [58]:
words_ontologia=[word[0] for word in most_similar_words]

#Palabras en plural o muy similares
more_words=[]
for word in words:
    additional = difflib.get_close_matches(word, diccionario_lemas.keys(), n=5, cutoff=0.90)
    more_words.extend(additional[1:])
    
#Añadir palabras finales
words_ontologia.extend(more_words)
# Recuperacion con OR2

words_instances=[]

for word in words_ontologia:
    instance = ontologia.search(descripcion_palabra=word)
    words_instances.extend(instance)

#print(words_instances)
docs = []
for word in words_instances:
    for doc in word.palabra_describe_pi:
        docs.append(doc)
        

# sorting on bais of frequency of elements 
# result_ontologia = list(chain(i for i, c in Counter(id_docs).most_common()))
result_ontologia = list(chain(i for i, c in Counter(docs).most_common())) 
id_ontologia = [id_pi.get_id_proyecto_investigacion()[0] for id_pi in result_ontologia]

  
# printing final result 
# print("final list", str(result), "Cantidad:",len(result)) 
for index, doc in enumerate(result_ontologia[:5]):
    title = doc.get_titulo_proyecto_investigacion()
    id_pi = doc.get_id_proyecto_investigacion()
    print(f'{id_pi[0]} - {title[0]}')

422 - MOTOR DE BÚSQUEDA INTELIGENTE DE INFORMES DE INVESTIGACIÓN BASADO EN RECURSOS SEMÁNTICOS PARA EL SISTEMA DE INVESTIGACIONES DE LA UNIVERSIDAD DE NARIÑO
443 - UMAYUX: UN MODELO DE SOFTWARE DE GESTION DE CONOCIMIENTO SOPORTADO EN UNA ONTOLOGIA DINAMICA DEBILMENTE ACOPLADO CON UN GESTOR DE BASE DE DATOS PARA LA UNIVERSIDAD DE NARIÑO
320 - APROXIMACIÓN AL PENSAMIENTO JURÍDICO NARIÑENSE DESDE LAS FIGURAS DE JOSÉ RAFAEL SAÑUDO Y ERNESTO VELA ANGULO.
438 - SITAPP: UNA APLICACIÓN INTELIGENTE PARA DISPOSITIVOS MÓVILES DEL SISTEMA DE RUTAS DE TRANSPORTE URBANO DEL MUNICIPIO DE PASTO
9 - Análisis de factores de aprendizaje en seguridad alimentaria con comunidades rurales en el municipio de Pasto, departamento de Nariño.


#### Consulta de Documentos mas similares y en contexto

In [123]:
' '.join(words)

'gestor conocimiento'

In [125]:
#Cantidad de Documentos 
limit_docs = len(id_ontologia)

infer_vec = [modeloD2V.infer_vector(words)]
vec_mean_doc = np.array(infer_vec).mean(axis=0)
unit_vec_doc = vec_mean_doc / np.linalg.norm(vec_mean_doc)

query_sims_docs = {
  "query": {
    "script_score": {
      "query": {
        "match_all": {}
       },
      "script": {
        "id": "dot_product",
        "params": {
          "query_vector": unit_vec_doc
        }
      }
    }
  },
  "size": limit_docs,
  "_source": ["tag"]
}

most_similar_docs = es.search(index='documentos', body=query_sims_docs)['hits']['hits']
most_similar_docs = [(doc['_source']['tag'], doc['_score']) for doc in most_similar_docs]
id_D2V = [id_pi[0] for id_pi in most_similar_docs]
most_similar_docs

[(443, 0.66235775),
 (438, 0.6223273),
 (422, 0.6201896),
 (417, 0.6085159),
 (435, 0.60569215),
 (139, 0.56078124),
 (337, 0.5443832),
 (306, 0.5406774),
 (65, 0.53958386),
 (425, 0.53944224),
 (444, 0.53565085),
 (445, 0.53490525),
 (21, 0.53432727),
 (407, 0.52386117),
 (311, 0.5203832),
 (316, 0.5185517),
 (197, 0.50988),
 (436, 0.5038429),
 (24, 0.50227654),
 (315, 0.49949726),
 (186, 0.49746048),
 (290, 0.49744958),
 (289, 0.49667418),
 (184, 0.49478057),
 (390, 0.4924835),
 (183, 0.49125305),
 (182, 0.49119693),
 (185, 0.49052423),
 (20, 0.4829776),
 (319, 0.4790032),
 (44, 0.47827286),
 (404, 0.47826412),
 (123, 0.47139874),
 (11, 0.47028577),
 (240, 0.46992493),
 (310, 0.46885532),
 (291, 0.46796596),
 (403, 0.4669853),
 (296, 0.46383485),
 (19, 0.46351784),
 (221, 0.46308765),
 (193, 0.46277997),
 (344, 0.4618602),
 (258, 0.46172166),
 (437, 0.4616627),
 (49, 0.46124566),
 (276, 0.45824677),
 (116, 0.45591488),
 (59, 0.45519447),
 (420, 0.4534743),
 (410, 0.45316967),
 (381, 

In [126]:
print(len(id_ontologia))
print(len(id_D2V))

165
165


## Comparacion de Resultados y Presentación Final

In [127]:
print(id_ontologia)
print(id_D2V)

[422, 443, 320, 438, 9, 34, 62, 63, 64, 190, 193, 204, 230, 231, 247, 248, 292, 293, 304, 311, 312, 337, 352, 375, 431, 433, 444, 1, 38, 281, 3, 4, 6, 7, 8, 10, 13, 14, 21, 24, 25, 28, 30, 39, 48, 49, 53, 55, 58, 65, 68, 73, 75, 77, 80, 83, 91, 94, 98, 103, 106, 107, 111, 117, 119, 122, 126, 127, 131, 143, 154, 157, 159, 160, 163, 168, 170, 177, 180, 188, 191, 196, 198, 201, 202, 205, 212, 214, 219, 220, 223, 224, 229, 232, 238, 251, 261, 263, 266, 271, 272, 276, 280, 282, 284, 286, 288, 291, 305, 308, 310, 321, 329, 331, 332, 334, 336, 338, 343, 345, 346, 349, 350, 360, 368, 371, 376, 377, 380, 382, 394, 405, 407, 409, 416, 417, 430, 441, 445, 448, 435, 104, 173, 197, 235, 255, 324, 395, 399, 400, 403, 12, 18, 47, 56, 71, 203, 218, 226, 233, 243, 245, 410, 419, 451]
[443, 438, 422, 417, 435, 139, 337, 306, 65, 425, 444, 445, 21, 407, 311, 316, 197, 436, 24, 315, 186, 290, 289, 184, 390, 183, 182, 185, 20, 319, 44, 404, 123, 11, 240, 310, 291, 403, 296, 19, 221, 193, 344, 258, 437, 49,

In [129]:
final_search = []
for onto, d2v in zip(id_ontologia, id_D2V):
    if onto == d2v:
        final_search.append(onto)
        continue
    else:
        if (onto not in final_search) and (d2v not in final_search):
            final_search.append(onto)
            final_search.append(d2v)
len(final_search)

225

In [115]:
type(ontologia)

owlready2.namespace.Ontology

In [131]:
final_result = [ontologia.search(id_proyecto_investigacion=id_pi)[0] for id_pi in final_search]
for pi in final_result:
    print(pi.get_titulo_proyecto_investigacion())


['MOTOR DE BÚSQUEDA INTELIGENTE DE INFORMES DE INVESTIGACIÓN BASADO EN RECURSOS SEMÁNTICOS PARA EL SISTEMA DE INVESTIGACIONES DE LA UNIVERSIDAD DE NARIÑO']
['UMAYUX: UN MODELO DE SOFTWARE DE GESTION DE CONOCIMIENTO SOPORTADO EN UNA ONTOLOGIA DINAMICA DEBILMENTE ACOPLADO CON UN GESTOR DE BASE DE DATOS PARA LA UNIVERSIDAD DE NARIÑO']
['SITAPP: UNA APLICACIÓN INTELIGENTE PARA DISPOSITIVOS MÓVILES DEL SISTEMA DE RUTAS DE TRANSPORTE URBANO DEL MUNICIPIO DE PASTO']
['MAKIPURAY: UNA HERRAMIENTA WEB DE INTELIGENCIA DE NEGOCIOS PARA EL DESCUBRIMIENTO DE CONOCIMIENTO ORIENTADA A LAS PYME DEL DEPARTAMENTO DE NARIÑO']
['Análisis de factores de aprendizaje en seguridad alimentaria con comunidades rurales en el municipio de Pasto, departamento de Nariño.']
['Rikhuna: Visor cartográfico inteligente de direcciones urbanas y sitios de interés del municipio de Pasto basado en PostGIS.']
['CARACTERIZACIÓN AEROBIA Y ANAEROBIA DEL ÁREA DE INFLUENCIA DE LA PRODUCCIÓN DE TRUCHA ARCOÍRIS (Oncorhynchus mykiss)

In [113]:
type(ontologia)

owlready2.namespace.Ontology